### Complex Schema Validation with Avro
**Description**: Implement a solution in Python to validate records against a complex nested Avro schema.

Eg., Complex schema ( nested_schema.avsc ):

**Steps**:
1. Load schema
2. Example data to validate
3. Validate against schema
4. Read back to check

In [4]:
import json
from fastavro import parse_schema, validate, writer, reader

# Step 1: Create the nested Avro schema file
schema_content = '''
{
  "type": "record",
  "name": "Patient",
  "fields": [
    {"name": "patient_id", "type": "string"},
    {"name": "name", "type": "string"},
    {"name": "age", "type": "int"},
    {
      "name": "address",
      "type": {
        "type": "record",
        "name": "Address",
        "fields": [
          {"name": "street", "type": "string"},
          {"name": "city", "type": "string"},
          {"name": "zip", "type": "string"}
        ]
      }
    },
    {
      "name": "appointments",
      "type": {
        "type": "array",
        "items": {
          "type": "record",
          "name": "Appointment",
          "fields": [
            {"name": "date", "type": "string"},
            {"name": "doctor", "type": "string"}
          ]
        }
      }
    }
  ]
}
'''

schema_file = "nested_schema.avsc"
with open(schema_file, "w") as f:
    f.write(schema_content)
print(f"✅ Created schema file '{schema_file}'")

# Step 2: Load schema
with open(schema_file, "r") as f:
    raw_schema = json.load(f)

schema = parse_schema(raw_schema)
print("✅ Loaded and parsed schema")

# Step 3: Example data to validate
example_records = [
    {
        "patient_id": "P001",
        "name": "John Doe",
        "age": 45,
        "address": {
            "street": "123 Main St",
            "city": "Anytown",
            "zip": "12345"
        },
        "appointments": [
            {"date": "2025-05-01", "doctor": "Dr. Smith"},
            {"date": "2025-06-15", "doctor": "Dr. Adams"}
        ]
    },
    {
        "patient_id": "P002",
        "name": "Jane Roe",
        "age": 38,
        "address": {
            "street": "456 Oak St",
            "city": "Othertown",
            "zip": "67890"
        },
        "appointments": [
            {"date": "2025-05-20", "doctor": "Dr. Clark"}
        ]
    }
]

# Step 4: Validate records against schema
print("\nValidating records...")
for i, record in enumerate(example_records, 1):
    is_valid = validate(record, schema)
    print(f" Record {i} validation result: {'Valid' if is_valid else 'Invalid'}")

# Step 5: Write records to Avro file
avro_filename = "patients.avro"
with open(avro_filename, "wb") as out:
    writer(out, schema, example_records)
print(f"\n✅ Written example records to '{avro_filename}'")

# Step 6: Read records back from Avro file and print
print("\nReading records back from Avro file:")
with open(avro_filename, "rb") as f:
    avro_reader = reader(f)
    for rec in avro_reader:
        print(rec)

✅ Created schema file 'nested_schema.avsc'
✅ Loaded and parsed schema

Validating records...
 Record 1 validation result: Valid
 Record 2 validation result: Valid

✅ Written example records to 'patients.avro'

Reading records back from Avro file:
{'patient_id': 'P001', 'name': 'John Doe', 'age': 45, 'address': {'street': '123 Main St', 'city': 'Anytown', 'zip': '12345'}, 'appointments': [{'date': '2025-05-01', 'doctor': 'Dr. Smith'}, {'date': '2025-06-15', 'doctor': 'Dr. Adams'}]}
{'patient_id': 'P002', 'name': 'Jane Roe', 'age': 38, 'address': {'street': '456 Oak St', 'city': 'Othertown', 'zip': '67890'}, 'appointments': [{'date': '2025-05-20', 'doctor': 'Dr. Clark'}]}
